In [222]:
from pymongo import MongoClient
import pprint

import pandas as pd
pd.set_option('display.max_rows', None)
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

import requests
from bs4 import BeautifulSoup

import json
import time
import copy

import os

from time import sleep
from selenium import webdriver

import sys
import random
import scipy.stats as scs
import numpy as np

import script

Step 1:
    gather data for three more hikes
    loop through data
    create dataframes
    join df's for 4 and 5 stars
    perform z-test

Step 2:
    perform further statistical tests
    gather more hike data?
    gather data on individual users and perform further tests

In [47]:
# Collecting hiking trail links from https://www.alltrails.com/explore in a particular hiking area
hiking_area_url = 'https://www.alltrails.com/explore/us/new-jersey/teaneck?b_tl_lat=40.95350100196839&b_tl_lng=-74.12586328125226&b_br_lat=40.84165171789468&b_br_lng=-73.90613671875113'

# Solves 403 'Forbidden' error
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'}

# getting html
r = requests.get(hiking_area_url, headers=headers)

# status_code check
if r.status_code != 200:
    print(r.status_code)

# parsing html
soup = BeautifulSoup(r.text, 'html.parser')

# getting hrefs to specific hikes in this area
hrefs = soup.find_all('a', itemprop="url")
hrefs

# .text.rstrip()

# find_all('a', itemprop="url" ... ('href')

# links = []
# for i, href in enumerate(hrefs):
#     cleaned_href = ('/').join(href.split('/')[2:])
#     links.append('https://www.alltrails.com/' + cleaned_href)

https://www.alltrails.com/trail/us/new-jersey/hackensack-river-county-park-trail
soup.find('div', class_='styles-module__container___10uYZ')

[]

In [141]:
# instantiating variables to be columns in dataframe
links = ['https://www.alltrails.com/trail/us/new-jersey/shore-trail-and-long-trail-loop-from-alpine-picnic-area', 'https://www.alltrails.com/trail/us/new-jersey/long-path-gw-bridge-to-lamont-doherty-earth-observatory']
written_review = ""
ratings = []
dates = []
types = []
review_text = []
reviewer_links = []
driver = webdriver.Chrome()

# Loop through links to get html, save to mongo, extract relevant data
for link in links:
    driver.get(link)
    sleep(3)

    driver.execute_script("window.scrollTo(0, 10000)")

# find 'show more reviews button'
    search = driver.find_element_by_css_selector("#reviews > div.styles-module__container___px-t2.xlate-none > button")
    
# prevents error being thrown once total trail reviews is reached; 
    showing_results = driver.find_element_by_css_selector('#reviews > div.styles-module__container___px-t2.xlate-none > div')
    total_ = int(showing_results.text.split(' ')[-1])
    per_page = int(showing_results.text.split(' ')[-3])

# clicks through to see additional reviews
    for _ in range((total_//per_page)-1):
        search.click()
        sleep(1)

# get page html
    html = driver.page_source
    time.sleep(2)

# add html to mongo
    client = MongoClient()
    db = client.alltrails
    trail_name = (link.split('/'))[-1]
    pages = db.trail_name
    pages.insert_one({'link': link ,'html': html})
    
# parsing data
    soup = BeautifulSoup(html, 'html.parser')

# getting trail title, description, difficulty, info
    soup.find('h1', class_='xlate-none styles-module__name___1nEtW').text.rstrip()
    soup.find('p', id="auto-overview").text.rstrip()
    soup.find('span', class_="styles-module__diff___22Qtv styles-module__moderate___3w1it styles-module__selected___3fawg").text.rstrip()
    soup.find('span', class_="styles-module__detailData___kQ-eK").text.rstrip()

# getting reviewer data for trail, appending to lists
    for i, review in enumerate(soup.find_all('div', itemprop="review")):

        ratings.append(soup.find_all('span', class_="MuiRating-root default-module__rating___1k45X MuiRating-sizeLarge MuiRating-readOnly")[i]['aria-label'])
        dates.append(soup.find_all('span', class_="styles-module__dateTrailDetails___3qgZC xlate-none")[i].text.rstrip())
        types.append(soup.find_all('span', class_="styles-module__tag___2s-oD styles-module__activityTag___3-RdN")[i].text.rstrip())

        written_review = soup.find_all('div', class_="styles-module__container___3etfA")[i].find('p', itemprop="reviewBody")

        if written_review == None:
            review_text.append(None)

        else:
            review_text.append(soup.find_all('div', class_="styles-module__container___3etfA")[i].find('p', itemprop="reviewBody").text.rstrip())        

#         reviewer_links.append('alltrails.com' + soup.find_all('div', class_="styles-module__container___3etfA")[1].find('a', class_="styles-module__link___2i6Za styles-module__recording___3jQX3 xlate-none")['href'])
    
    # for trail in find('ul', class_=" float-delete-button").find_all('li', class_="styles-module__container___10uYZ"):

In [ ]:
, 'https://www.alltrails.com/trail/us/new-jersey/henry-hudson-drive', 'https://www.alltrails.com/trail/us/new-jersey/flat-rock-brook-loop-trail']

In [152]:
# close Selenium ChromeDriver
driver.close()

# collecting data into dataframe
df = pd.DataFrame({'rating': ratings, 'date':dates, 'desc': types, 'reviews': review_text}).sort_values('rating') # to be added: 'links': reviewer_links

### Frequentist Approach

In [153]:
five = df[df['rating'] == '5 Stars']['reviews'].count() #num of written reviews left by five-star reviewers
four = df[df['rating'] == '4 Stars']['reviews'].count() #num of written reviews left by four-star reviewers
num_five = df[df['rating']=='5 Stars']['rating'].count() #num of five-star reviewers
num_four = df[df['rating']=='4 Stars']['rating'].count() #num of four-star reviewers

table = pd.DataFrame({'Written Reviews': [four, five], 'Total Reviewers': [num_four, num_five]}, index = ['4 Stars', '5 Stars']).T
table = table.style.set_properties(**{'text-align': 'center'})
table

,4 Stars,5 Stars
Written Reviews,47,75
Total Reviewers,206,361


At first glance it is obvious that a much higher percentage of 4 star reviewers left written reviews than 5 star reviewers.

## Hypotheses

H0: Mu4 = Mu5 (4 and 5 star reviewers are equally likely to leave written reviews — P(R | 4) = P(R | 5))

Ha: Mu4 != Mu5 (4 and 5 star reviewers leave written reviews at different rates — P(R | 4) != P(R | 5))

Test Statistic: 4-Stars: 206 reviews and 5-Stars: 361 reviewers

Alpha: 0.05

In [226]:
four_stars_distribution = ([0] * (206 - 47)) + ([1] * 47)
five_stars_distribution = (([0] * (361 - 75)) + ([1] * 75))
four_stars_prob_distribution = []
five_stars_prob_distribution = []
for i in four_stars_distribution:
    four_stars_prob_distribution.append((mean_pop, 1 - mean_pop)[i])
for i in five_stars_distribution:
    five_stars_prob_distribution.append((mean_pop, 1 - mean_pop)[i])

In [227]:
# calculate welch test statistic
test_statistic = welch_test_statistic(sample_1, sample_2)

In [228]:
# calculate degrees of freedom
df = welch_satterhwaithe_df(sample_1, sample_2)

# set up distribution
students = scs.t(df)

# calculate p-value
p_value = 1 - students.cdf(test_statistic)
p_value

0.2867215735365761

Fail to reject the H0.

In conclusion, the data fails to show a difference in the average number of written reviews left by 4-star reviewers and 5-star reviewers.